# Data Crawl and Clean
---
Yufan Zhang (yz605@duke.edu)

Junyi Tao (jt328@duke.edu)

Ran Peng (rp231@duke.edu)

In [34]:
import requests
import json
import os
import pandas as pd
import numpy as np
import re
import pickle

## 1. Use DKU Faculty Members' emails to batch search for DUIDs

Duke  IdM Tools-[LDAP Batch Search by Identifer](https://idm.oit.duke.edu/tools/ldap)

In [2]:
dku_emails = pd.read_csv('../data/dku_emails.csv')

s_temp = ''
for i in range(len(dku_emails)):
    s_temp += dku_emails.iloc[i]['Email']+'\n'
    
with open('../data/dku_duid_list.txt','w') as f:
    f.write(s_temp)

In [3]:
LDAP_data = pd.read_csv('../data/2022-4-11-ldap.csv')
LDAP_data.head()

,dn,Duke UniqueID (duDukeID),NetID (uid),First Name (givenName),Last Name (sn),Common Name (cn),Display Name (dn),Nickname (eduPersonNickname),Title (title),Affiliation(s) (eduPersonAffiliation),Primary Affiliation (eduPersonPrimaryAffiliation),Email Address (mail),Telephone Number (telephoneNumber),Company Code Description (duSAPCompanyDesc),Organizational Unit (ou),SAP Org Unit (duSAPOrgUnit),Email Reciept and Telephone Number Privacy Flag (duUserPrivacyRequest),Postal Address (PostalAddress),Post Office Box (PostOfficeBox),Generational Name Qualifier (generationQualifier)
0,duLDAPKey=dc808dee-1dd1-11b2-b6a9-9442cd793b76...,480813,djs26,Daniel,Stephens,Daniel Stephens Daniel J Stephens,"Daniel Stephens, Ph.D.",Daniel,Assistant Professor of Philosophy at Duke Kuns...,alumni affiliate faculty,faculty,daniel.stephens@duke.edu,+1 919 668 0377,Duke University,Dku Appointments,51032158,NaN,NaN,NaN,NaN
1,duLDAPKey=3a750dd4-1dd2-11b2-8a54-9442cd793b76...,501216,jc254,Jung,Choi,Jung Choi Jung E Choi,"Prof Jung Choi, Ph.D.",Jung,Assistant Professor of Arts History and Visual...,alumni affiliate,affiliate,jung.choi@dukekunshan.edu.cn,+1 919 684 2224,Duke University,NaN,51032158,NaN,NaN,NaN,NaN
2,duLDAPKey=0f2ac190-1dd2-11b2-b7bc-e02c52381c12...,663997,ba83,Benjamin,Anderson,Benjamin Anderson,Benjamin Anderson,Benjamin,Assistant Professor of Global Health at Duke K...,affiliate,affiliate,benjamin.anderson2@dukekunshan.edu.cn,NaN,Duke University,NaN,51032158,NaN,NaN,NaN,NaN
3,duLDAPKey=7ff6272e-1dd2-11b2-b7c3-e02c52381c12...,664012,dbs25,Donald,Snow,Donald Snow Don Snow,Don Snow,Don,Director of Language and Writing Program at Du...,affiliate,affiliate,don.snow@dukekunshan.edu.cn,NaN,Duke University,NaN,51032158,NaN,NaN,NaN,NaN
4,duLDAPKey=e456d7dc-f85d-11e5-a18d-c3e21a144c1c...,715779,kmc104,Kaley,Clements,Kaley Clements,Kaley Clements,Kaley,Assistant Professor of Media and Arts at Duke ...,alumni affiliate faculty,faculty,kaley.clements@duke.edu,+1 919 684 1958,Duke University,Dku Appointments,51032158,NaN,NaN,NaN,NaN


In [4]:
duids_dku = LDAP_data[['Duke UniqueID (duDukeID)', 'Common Name (cn)']]
duids_dku.columns = ['id', 'name']
duids_dku = duids_dku[['name', 'id']]
duids_dku.to_csv('../data/duids_dku.csv')
duids_dku.head()

,name,id
0,Daniel Stephens Daniel J Stephens,480813
1,Jung Choi Jung E Choi,501216
2,Benjamin Anderson,663997
3,Donald Snow Don Snow,664012
4,Kaley Clements,715779


## 2. Use the DUIDs to crawl JSON via Scholars widgets API

[Scholars@Duke Swagger widgets API](https://scholars.duke.edu/widgets/docs/#!/widgets/get_api_v0_9_people_complete_count_format)

### Load DUIDs

In [5]:
duids_all = pd.read_csv('../data/duids_all.csv')
duids_all = duids_all[pd.notnull(duids_all['id'])].id.apply(lambda x: str(int(x)).zfill(7))
all = tuple(duids_all)
print(len(all))

# duids_dku = pd.read_csv('../data/duids_dku.csv')
duids_dku = duids_dku[pd.notnull(duids_dku['id'])].id.apply(lambda x: str(int(x)).zfill(7))
dku = tuple(duids_dku)
print(len(dku))

duids = list(set(all + dku))
print(len(duids))

6927
104
6910


In [6]:
duids

['0592191',
 '0031274',
 '0496961',
 '0074043',
 '0631709',
 '0593942',
 '1039889',
 '0519917',
 '0280189',
 '0175258',
 '0096830',
 '0907974',
 '0393319',
 '0098664',
 '0028214',
 '0636946',
 '0230411',
 '0284638',
 '0109375',
 '0583478',
 '0113731',
 '0731317',
 '0377213',
 '0382769',
 '0685081',
 '0038207',
 '0565437',
 '0098755',
 '0810207',
 '0685672',
 '0311671',
 '0302045',
 '0061790',
 '0275535',
 '0263061',
 '0121483',
 '1040914',
 '0114012',
 '0030443',
 '0219488',
 '0281146',
 '1114684',
 '0686657',
 '0309896',
 '0116251',
 '0538635',
 '0630772',
 '0304061',
 '0668184',
 '0545187',
 '0490614',
 '0497156',
 '0376336',
 '0600522',
 '0603030',
 '0502936',
 '0802246',
 '0452688',
 '0097644',
 '0168671',
 '0907118',
 '1114657',
 '0120199',
 '0492120',
 '0312267',
 '0276392',
 '0378319',
 '0982052',
 '0564560',
 '0915857',
 '0310370',
 '0980812',
 '1162109',
 '0800652',
 '0455903',
 '0196593',
 '1227549',
 '0229181',
 '0541996',
 '0101137',
 '0810821',
 '0116986',
 '1057410',
 '10

### Crawl JSON data

Data Mode
- Complete
- Publication

In [7]:
def crawl(duid, data_mode):
    url = 'https://scholars.duke.edu/widgets/api/v0.9/people/{}/5.json?uri={}'.format(data_mode, duid)
    headers = {"Accepct": "application/json"}
    response = requests.get(url=url, headers=headers)
    if response.status_code == 404:
        return ('No Response', duid)
    json_info = response.json()
    return json_info

def save_json(data_mode, duid, content):
    dir_str = '../data/scholars_duke_new/{}/{}.json'.format(data_mode, duid)
    with open(dir_str, 'w') as outfile:
        json.dump(content, outfile)

In [8]:
def get_all_json(duids, data_mode):
    '''
        duids: list of duids
        data_mode: 'complete', 'publications', 'courses', 'educations'
    '''
    counter = 0
    for i in range(len(duids)):
        duid = duids[i]
        # name = duid_df.loc[i, 'name']
        no_info_list = list()
        
        json_content = crawl(duid, data_mode)
            
        if type(json_content) == tuple:
            no_info_list.append(json_content[1])
        else:
            counter += 1
            save_json(data_mode, duid, json_content)
        
        if i % 500 == 0:
            print('============== {}/{} completed =============='.format(i, len(duids)))
    print('============== {} JSON files containing {} infomation in total crawled =============='.format(counter, data_mode))

In [45]:
get_all_json(duids, 'complete')

============== 0/6910 completed ==============
============== 500/6910 completed ==============
============== 1000/6910 completed ==============
============== 1500/6910 completed ==============
============== 2000/6910 completed ==============
============== 2500/6910 completed ==============
============== 3000/6910 completed ==============
============== 3500/6910 completed ==============
============== 4000/6910 completed ==============
============== 4500/6910 completed ==============
============== 5000/6910 completed ==============
============== 5500/6910 completed ==============
============== 6000/6910 completed ==============
============== 6500/6910 completed ==============
============== 6801 JSON files containing complete infomation in total crawled ==============


## 3. Clean the JSON files

In [9]:
def get_education(d):
    '''
        Input:
            d: a dict containing the education information, ori_d['educations'][i]
        Output:
            a dict containing the degree,institution, end_year
    '''
    out = dict()
    if 'degree' in d['attributes'].keys():
        out['degree'] = d['attributes']['degree']
    else:
        out['degree'] = None
    
    if 'institution' in d['attributes'].keys():
        out['institution'] = d['attributes']['institution']
    else:
        out['institution'] = None
        
    if 'endDate' in d['attributes'].keys():
        out['end_year'] = d['attributes']['endDate']
    else:
        out['end_year'] = None
    return out


def get_publication(d):
    '''
        Input:
            d: a dict containing the publication information, ori_d['publications'][i]
        Output:
            a dict containing the title, authors, year
    '''
    out = dict()
    out['title'] = d['label']
    if 'authorList' not in d['attributes'].keys():
        out['authors'] = None
    else:
        out['authors'] = d['attributes']['authorList']
        
    if 'year' not in d['attributes'].keys():
        out['year'] = None
    else:
        out['year'] = d['attributes']['year']
    return out


def get_courses(d):
    '''
        Input:
            d: a dict containing the courses information, ori_d['courses'][i]
        Output:
            a string containing the course_label
    '''
    out = d['label']
    return out

In [18]:
def get_info(json_file_name):
    '''
        Input:
            json_file_name: the name of the json file
        Output:
            a dict containing faculty information, counts for the educations, publications and courses
    '''
    f = open('../data/scholars_duke_new/complete/' + json_file_name, 'r')
    raw_d = dict(json.load(f))
    ind_d = dict()
    duid = json_file_name.split('.')[0]
    
    ind_d['duid'] = duid                                                    # duid
    ind_d['name'] = raw_d['label']                                          # name
    ind_d['uri'] = raw_d['uri']                                             # uri
    ind_d['title'] = raw_d['title']                                         # title
    
    from_dku = False
    if len(raw_d['positions']) > 0:
        ind_d['school'] = raw_d['positions'][0]['attributes']['schoolLabel']        # school
        for pos in raw_d['positions']:
            if pos['attributes']['schoolLabel'] == 'Duke Kunshan University':
                from_dku = True     
    else:
        ind_d['school'] = None
    
    educations = list()
    for edu_d in raw_d['educations']:
        educations.append(get_education(edu_d))
    ind_d['educations'] = educations                                       # educations
        
    publications = list()
    for pub_d in raw_d['publications']:
        publications.append(get_publication(pub_d))
    ind_d['publications'] = publications                                   # publications
    
    courses = list()
    for cou_d in raw_d['courses']:
        courses.append(get_courses(cou_d))
    ind_d['course'] = courses                                              # courses
    
    return ind_d, len(educations), len(publications), len(courses), from_dku

In [19]:
def clean_json():
    '''
    Output:
        a dictionary containing all the information
        {duid : {   duid : string, 
                    name : string,
                    uri : string, 
                    title : string,
                    school : string,
                    educations : [
                        {
                            degree: string,
                            institution : string,
                            end_year : string
                        }
                    ], 
                    publications : [
                        {
                            title : string,
                            authors : string,
                            year : string
                        }
                    ]
                    courses : [
                        string
                    ]
                }
            }
    '''
    out_d = dict()
    out_d_dku = dict()
    json_file_names = os.listdir('../data/scholars_duke_new/complete/')
    counter = 0
    edu_counter, pub_counter, cou_counter = 0, 0, 0
    edu_sum, pub_sum, cou_sum = 0, 0, 0
    
    for json_file_name in json_file_names:
        if re.match(r"[0-9]+(.json)", json_file_name):
            # print('=== Processing {} ==='.format(json_file_name))
            
            duid = json_file_name.split('.')[0]
            
            # Get individual information
            ind_d, n_edu, n_pub, n_cou, from_dku = get_info(json_file_name)
            
            ## save to out_d_dku if the person is a DKU faculty
            if from_dku:
                out_d_dku[duid] = ind_d
                
            out_d[duid] = ind_d
            
            # Log counter
            if n_edu > 0:
                edu_counter += 1
                edu_sum += n_edu
            if n_pub > 0:
                pub_counter += 1
                pub_sum += n_pub
            if n_cou > 0:
                cou_counter += 1
                cou_sum += n_cou
            counter += 1
            
            if counter % 500 == 0:
                print('============= {}/{} ============='.format(counter, len(json_file_names)))
    
    print('=== Summary ===')   
    print('count of scholars who have educations info: {}/{}'.format(edu_counter, len(json_file_names)))
    print('total education info pieces: {}\n'.format(edu_sum))
    print('count of scholars who have publications info: {}/{}'.format(pub_counter, len(json_file_names)))
    print('total publication info pieces: {}\n'.format(pub_sum))
    print('count of scholars who have courses info: {}/{}'.format(cou_counter, len(json_file_names)))
    print('total course info pieces: {}\n'.format(cou_sum))
    
    print('DKU/Duke: {}/{}'.format(len(out_d_dku), len(out_d)))
    
    with open("../cleaned_scholars.json", "w") as outfile:
        json.dump(out_d, outfile)
    print('=== All Info: Done! ===')
    
    with open("../cleaned_scholars_dku.json", "w") as outfile:
        json.dump(out_d_dku, outfile)
    print('=== DKU Info: Done! ===')
    
    return out_d, out_d_dku, [edu_counter, pub_counter, cou_counter], [edu_sum, pub_sum, cou_sum]

In [20]:
out_d, out_d_dku, counters, sums = clean_json()

============= 500/6801 =============
============= 1000/6801 =============
============= 1500/6801 =============
============= 2000/6801 =============
============= 2500/6801 =============
============= 3000/6801 =============
============= 3500/6801 =============
============= 4000/6801 =============
============= 4500/6801 =============
============= 5000/6801 =============
============= 5500/6801 =============
============= 6000/6801 =============
============= 6500/6801 =============
=== Summary ===
count of scholars who have educations info: 6665/6801
total education info pieces: 14403

count of scholars who have publications info: 4425/6801
total publication info pieces: 374192

count of scholars who have courses info: 2493/6801
total course info pieces: 15268

DKU/Duke: 184/6801
=== All Info: Done! ===
=== DKU Info: Done! ===


## 4. Filter the educations, publications, and courses info for DKU faculty

In [21]:
with open("../cleaned_scholars_dku.json") as f:
    cleaned_dku = json.load(f)

In [49]:
cleaned_dku['0729790']['educations']

[{'degree': 'MR',
  'institution': 'Fudan University (China)',
  'end_year': '2001-01-01T00:00:00'},
 {'degree': 'Ph.D.',
  'institution': 'Carnegie Mellon University',
  'end_year': '2005-01-01T00:00:00'}]

In [30]:
def extract_to_list(d):
    duid = d['duid']
    name = d['name']
    title = d['title']
    educations = d['educations']
    pub_titles = list()
    for pub in d['publications']:
        pub_titles.append(pub['title'])
    courses = d['course']
    return [duid, name, title, pub_titles, courses]

In [31]:
def extract_to_nested_list(school):
    if school == 'dku':
        with open("../cleaned_scholars_dku.json") as f:
            cleaned = json.load(f)
    elif school == 'duke':
        with open("../cleaned_scholars.json") as f:
            cleaned = json.load(f)
    else:
        print('Wrong school name!')
        return
    result_list = list()
    for ind_d_key in cleaned:
        result_list.append(extract_to_list(cleaned[ind_d_key]))
    print('=== {}: {} in total ==='.format(school, len(result_list)))
    return result_list

In [88]:
result_list_dku = extract_to_nested_list('dku')
# with (open('../data/nested_list_dku.pkl', 'wb')) as f:
#     pickle.dump(result_list, f)
result_df_dku = pd.DataFrame(result_list_dku, columns=['duid', 'name', 'title', 'pub_titles', 'courses'])
result_df_dku.to_csv('../data/nested_list_dku.csv', index=False)

=== dku: 184 in total ===


In [51]:
result_list = extract_to_nested_list('duke')
result_df = pd.DataFrame(result_list, columns=['duid', 'name', 'title', 'pub_titles', 'courses'])
result_df.to_csv('../data/nested_list_duke.txt', index=False)

=== duke: 6801 in total ===


In [65]:
result_df.title

0       Consulting Professor in the Department of Medi...
1       Consulting Associate in the Department of Fami...
2                                    Professor of Surgery
3            Adjunct Associate Professor of Global Health
4        Clinical Associate in the Department of Medicine
                              ...                        
6796    Adjunct Associate Professor in the Department ...
6797             Professor of Psychology and Neuroscience
6798        Consulting Associate in the School of Nursing
6799      William W. Elliott Assistant Research Professor
6800                      Assistant Professor of Medicine
Name: title, Length: 6801, dtype: object

In [153]:
def check_department(x):
    if re.match(r'.*(Professor) [(in)|(of)].*', x):
        return True
    elif re.match(r'.*(Lecturer) [(in)|(of)].*', x):
        return True
    else:
        return False

def extract_department(x):
    # Assistant Professor of Mathematics at Duke Kunshan University
    if re.match(r'.*(Professor of).*', x):
        return x.split('Professor of')[1].split(' at ')[0].strip()
    
    # Professor in the Department of Electrical and Computer Engineering
    elif re.match(r'.*(Professor in the Department).*', x):
        return x.split('Professor in the Department of ')[1].split(' at ')[0].strip()
        
    # Associate Professor in Environmental Sciences and Policy
    elif re.match(r'.*(Professor in).*', x):
        return x.split('Professor in')[1].split(' at ')[0].strip()
    
    # Assoc. Prof. of Cognitive Neuroscience at Duke Kunshan University
    elif re.match(r'.*(Prof. of).*', x):
        return x.split('Prof. of')[1].split(' at ')[0].strip()
    
    # Assistant Professorof Religious Studies at Duke Kunshan University
    elif re.match(r'.*(Professorof).*', x):
        return x.split('Professorof')[1].split(' at ')[0].strip()
    
    # Lecturer of Chinese Language at Duke Kunshan University
    elif re.match(r'.*(Lecturer of).*', x):
        return x.split('Lecturer of')[1].split(' at ')[0].strip()
        
    # Senior Lecturer in Physics at Duke Kunshan University
    elif re.match(r'.*(Lecturer in).*', x):
        return x.split('Lecturer in')[1].split(' at ')[0].strip()

    # Director of Athletic Department at Duke Kunshan University
    elif re.match(r'.*(Director of).*', x):
        return x.split('Director of')[1].split(' at ')[0].strip()
    
    # Visiting Instructor of English Language at Duke Kunshan University
    elif re.match(r'.*(Instructor of).*', x):
        return x.split('Instructor of')[1].split(' at ')[0].strip()
    
    # Professor
    elif x == 'Professor':
        return 'Professor'
    
    # University Librarian at Duke Kunshan University
    else:
        return x.split(' at ')[0].strip()


def get_department(result_df, save_dir):
    result_df['department'] = result_df.title.apply(extract_department)
    result_df.to_csv(save_dir)

In [154]:
result_df_dku = pd.DataFrame(result_list_dku, columns=['duid', 'name', 'title', 'pub_titles', 'courses'])
get_department(result_df_dku, '../data/info_dku_department.txt')

In [156]:
result_df = pd.DataFrame(result_list, columns=['duid', 'name', 'title', 'pub_titles', 'courses'])
get_department(result_df, '../data/info_duke_department.txt')

In [158]:
result_df.to_csv('test2.csv')

In [62]:
result_df['have_in_of'] = result_df.apply(lambda x: not('in' in x['title'] or 'of' in x['title']), axis=1)
print(result_df[result_df['have_in_of'] == True].shape)
result_df[result_df['have_in_of'] == True]

(25, 6)


,duid,name,title,pub_titles,courses,have_in_of
83,0501633,"Huang, Ryan",Temporary Instructor,[],"[IDS 798: Capstone Project, STA 798: Capston...",True
229,0808074,"Isabella, Zoe",Instructor B,[],[BIOLOGY 202L: Gateway to Biology: Genetics a...,True
275,0746148,"Zhang, Chen",Student,[],[],True
954,0745840,"Yu, Handi",Student,[],[],True
1313,0898135,"Goatley, David Emmanuel",Associate Dean for Vocational Formation and Ch...,[Were You There? Godforsakenness in Slave Reli...,[BCS 760: Theology in the Black Church Tradit...,True
1395,0550579,"Friedmann, Victor",Instructor,[],[],True
2175,0061257,"Skender, Charles",Instructor,[Using a bankruptcy model in the auditing cour...,[ACCOUNTG 596: Accounting for Mergers and Acq...,True
2274,0587377,"Perera, Nadeesha",Instructor B,[],"[BIOLOGY 490S: Special Topics Seminar, K_BIOL...",True
2518,0379166,"Fay, John Patrick",Instructor,[Using geographic information systems to chara...,[ENVIRON 593: Independent Studies and Project...,True
2630,0415082,"Ozdowski, Emily",Instructor,[],[BIOLOGY 329D: Principles of Animal Physiolog...,True


,duid,name,title,pub_titles,courses,have_in_of
83,0501633,"Huang, Ryan",Temporary Instructor,[],"[IDS 798: Capstone Project, STA 798: Capston...",True
229,0808074,"Isabella, Zoe",Instructor B,[],[BIOLOGY 202L: Gateway to Biology: Genetics a...,True
275,0746148,"Zhang, Chen",Student,[],[],True
954,0745840,"Yu, Handi",Student,[],[],True
1313,0898135,"Goatley, David Emmanuel",Associate Dean for Vocational Formation and Ch...,[Were You There? Godforsakenness in Slave Reli...,[BCS 760: Theology in the Black Church Tradit...,True
1395,0550579,"Friedmann, Victor",Instructor,[],[],True
2175,0061257,"Skender, Charles",Instructor,[Using a bankruptcy model in the auditing cour...,[ACCOUNTG 596: Accounting for Mergers and Acq...,True
2274,0587377,"Perera, Nadeesha",Instructor B,[],"[BIOLOGY 490S: Special Topics Seminar, K_BIOL...",True
2518,0379166,"Fay, John Patrick",Instructor,[Using geographic information systems to chara...,[ENVIRON 593: Independent Studies and Project...,True
2630,0415082,"Ozdowski, Emily",Instructor,[],[BIOLOGY 329D: Principles of Animal Physiolog...,True
